In [1]:
import sys, os, time
# os.environ['PYTHONHASHSEED'] = '0'
import random
import numpy as np
seed = 10
RNG = random.Random(seed)
random.seed(seed)
np.random.seed(seed)

import util as util
from util import adict, idict, xset, collate, load, render_dict, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
from itertools import chain, product
from tnt_units import load_unit_rules
import tnt_setup as setup

from flask_app import *

print(ping())

# out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Backend active: use "init" to init game


In [2]:
out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Set seed 10
Beginning phase: Setup
Game saved
{({'Munich'}, {'Infantry', 'Tank', 'AirForce', 'Fortress'}), ({'Berlin', 'Ruhr', 'Konigsberg'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}
{({'Tripoli', 'Rome', 'Milan'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}
{({'Moscow', 'Urals', 'Stalingrad', 'Kiev'}, {'Infantry', 'Tank', 'AirForce', 'Fortress'}), ({'Baku', 'Odessa', 'Leningrad', 'Kharkov'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}
{({'Delhi', 'Bombay', 'Suez', 'Glasgow', 'London'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}
{({'Algiers', 'Paris', 'Marseille'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}
adict(Axis:{('Italy', {({'Tripoli', 'Rome', 'Milan'}, {'Fleet', 'Infantry', 'Carrier', 'Submarine', 'AirForce', 'Fortress', 'Tank'})}), ('Germany', {({'Munich'}, {'Infantry', 'Tank', 'AirForce', 'Fort

In [2]:
out = format_msg_to_python(init_game(debug=True, player='Axis', seed=seed))

Set seed 10
Beginning phase: Setup
Game saved
{({'Munich'}, {'Fortress', 'Tank', 'Infantry', 'AirForce'}), ({'Ruhr', 'Berlin', 'Konigsberg'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}
{({'Rome', 'Tripoli', 'Milan'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}
{({'Urals', 'Kiev', 'Stalingrad', 'Moscow'}, {'Fortress', 'Tank', 'Infantry', 'AirForce'}), ({'Leningrad', 'Baku', 'Odessa', 'Kharkov'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}
{({'Bombay', 'Delhi', 'Suez', 'London', 'Glasgow'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}
{({'Marseille', 'Paris', 'Algiers'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}
adict(Axis:{('Italy', {({'Rome', 'Tripoli', 'Milan'}, {'Tank', 'AirForce', 'Submarine', 'Carrier', 'Fleet', 'Infantry', 'Fortress'})}), ('Germany', {({'Munich'}, {'Fortress', 'Tank', 'Infantry', 'AirF

In [4]:
G = get_G()
# render_dict(G)

In [5]:
now = out.actions#list(util.decode_actions(out.actions))

In [6]:
print(now)

{('Germany', {({'Ruhr', 'Konigsberg', 'Berlin'}, {'Carrier', 'Tank', 'AirForce', 'Fleet', 'Infantry', 'Fortress', 'Submarine'}), ({'Munich'}, {'Tank', 'AirForce', 'Infantry', 'Fortress'})}), ('Italy', {({'Milan', 'Rome', 'Tripoli'}, {'Carrier', 'Tank', 'AirForce', 'Fleet', 'Infantry', 'Fortress', 'Submarine'})})}


In [7]:
print(prev)

NameError: name 'prev' is not defined

In [ ]:
if prev != str(now):
    print(now)
    print(prev)
prev = now

In [ ]:
now

In [ ]:
if now != prev:
    print('failed')
prev = now

In [ ]:
for x, y in zip(now,prev):
    if x != y:
        print(x, y)
        print('failed')
prev = now